In [2]:
%cd ..

/data/yantan/guwen-benchmark


In [256]:
import unicodedata


task_name = 'quote'
# preds_path = f'output/chinese-roberta-wwm/{task_name}/test_predictions.txt'
preds_path = f'output/guwenbert/{task_name}-2/test_predictions.txt'
datasets_path = 'data/punctuation/all.test.txt'


with open(datasets_path, 'rt') as f:
    labels = []
    lines = []
    quote_tags = []
    tokens = []
    for line in f:
        if line.startswith("-DOCSTART-") or line == "" or line == "\n":
            if quote_tags:
                lines.append(''.join(tokens))
                labels.append(quote_tags)
                quote_tags = []
                tokens = []
        else:
            splits = line.split(" ")
            char = splits[0]
            cat = unicodedata.category(char)
            if cat.startswith("C"):
                continue
            quote_tags.append(splits[3])
            tokens.append(char)
    labels.append(quote_tags)
    lines.append(''.join(tokens))

with open(preds_path, 'rt') as f:
    preds = f.read().split('\n')
    preds = [l.split() for l in preds if l]
    


In [257]:
for i, (line, l, p) in enumerate(zip(lines, labels, preds)):
    assert len(l) == len(p), (line, i, len(l), len(p))

In [258]:
from seqeval.metrics import f1_score, precision_score, recall_score


In [259]:
cats = {'易藏':36,'医藏':261,'艺藏':20,'史藏':1195,'佛藏':30,'集藏':1679,'诗藏':253,'子藏':412,'儒藏':80,'道藏':26}
start = 0
result = []
for cat in cats:
    cat_len = cats[cat]
    cat_preds = preds[start:start + cat_len]
    cat_labels = labels[start:start + cat_len]
    result.append([cat, precision_score(cat_labels, cat_preds), recall_score(cat_labels, cat_preds), f1_score(cat_labels, cat_preds)])
    start += cat_len
result.append(['all', precision_score(labels, preds), recall_score(labels, preds), f1_score(labels, preds)])

In [260]:
from pandas import DataFrame

In [261]:
df = DataFrame(result).transpose()

In [262]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,易藏,医藏,艺藏,史藏,佛藏,集藏,诗藏,子藏,儒藏,道藏,all
1,0.0,0.166667,0.789474,0.469697,0.461538,0.773282,0.745283,0.477024,0.825,0.636364,0.637926
2,0.0,0.16,0.75,0.657576,0.75,0.791064,0.67234,0.687697,0.785714,0.777778,0.730756
3,0.0,0.163265,0.769231,0.54798,0.571429,0.782072,0.706935,0.563307,0.804878,0.7,0.681193
